In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#%matplotlib inline
import time
from datetime import datetime
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import r, numpy2ri, FloatVector, pandas2ri
from datetime import timedelta
from IPython import get_ipython
import sys

# Observation

## 定義所採納的時間

In [2]:

start = str(sys.argv[1]) + ' 00:00:00' #input('起始日期：(格式:2019-01-01)')
end = str(sys.argv[2]) + ' 23:00:00' #input('結束日期：(格式:2019-10-17)')
time_step = int(sys.argv[3]) #int(input('預報小時：(格式:96)'))
ver = str(sys.argv[4]).split(",") #[int(num) for num in input("請輸入全部的版本(格式：17 18),以空白鍵區隔：").split(",")] 


'\nstart = str(sys.argv[1]) + \' 00:00:00\' #input(\'起始日期：(格式:2019-01-01)\')\nend = str(sys.argv[2]) + \' 23:00:00\' #input(\'結束日期：(格式:2019-10-17)\')\ntime_step = int(sys.argv[3]) #int(input(\'預報小時：(格式:96)\'))\nver = str(sys.argv[4]).split(",") #[int(num) for num in input("請輸入全部的版本(格式：17 18),以空白鍵區隔：").split(",")] \n'

In [4]:
def data_cleaning(df):
    
    df['year'] = df['year'].astype(str)
    df['month'] = df['month'].astype(str)
    df['day'] = df['day'].astype(str)
    df['hour'] = df['hour'].astype(str)
    df['dateTime'] = df['year'] + '-' + df['month'].str.zfill(2) + '-' + df['day'].str.zfill(2) + ' ' + df['hour'].str.zfill(2) +':'+ '00:00 '
    df['dateTime'] = pd.to_datetime(df['dateTime']).apply(lambda x:int(time.mktime(x.timetuple())))
    df.sort_values(by=['dateTime', 'stationID'], inplace = True)

    
    return df

In [5]:
time_interval = time_step // 24
start = time.strptime(start, "%Y-%m-%d %H:%M:%S")
end = time.strptime(end, "%Y-%m-%d %H:%M:%S")

# 轉換為時間戳:
start = int(time.mktime(start)) - 3600*24
end_pre = int(time.mktime(end))
en_ob = int(time.mktime(end)) 

In [6]:
new_observation = pd.read_table("opendata_epa.txt") 
new_observation = data_cleaning(new_observation)
file_name = 'corr_old_opendata_epa_' + str(time_step) + '.txt'
try:
    old_observation = pd.read_table(file_name)
    old_observation = data_cleaning(old_observation)
    old_end = old_observation['dateTime'].iloc[-1] + 3600
    new_observation = new_observation[new_observation['dateTime']>=old_end]
    df_observation = pd.concat([old_observation, new_observation], axis = 0, ignore_index=True)
    print('舊檔新檔合併')
except IOError:
    df_observation = new_observation
    print('讀取新檔')

讀取新檔


In [7]:
df_observation = df_observation[df_observation['dateTime'].between(start - 3600 * 24, en_ob + (time_interval) * 3600 * 24)]
df_observation['dateTime'] = df_observation['dateTime'].apply(lambda x:time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(x)))
df_observation.sort_values(by=['dateTime', 'stationID'], inplace = True)
opendata_station = list(pd.unique(df_observation['stationID']))

df_observation.tail()

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,year,month,day,hour,stationID,PMf,dateTime
553571,2019,10,31,23,EPA084,NaN,2019-10-31 23:00:00
553572,2019,10,31,23,EPA101,22.0,2019-10-31 23:00:00
553573,2019,10,31,23,EPA102,19.0,2019-10-31 23:00:00
553574,2019,10,31,23,EPA104,22.0,2019-10-31 23:00:00
553575,2019,10,31,23,EPA110,27.0,2019-10-31 23:00:00


## 移除測站'EPA101',' EPA084',' EPA104'(因為全部皆為NAN)

In [8]:
#pd.value_counts(df['stationID'])
df_observation = df_observation[df_observation['stationID']!='EPA101']
df_observation = df_observation[df_observation['stationID']!='EPA084']
df_observation = df_observation[df_observation['stationID']!='EPA104']

df_observation.head()

,year,month,day,hour,stationID,PMf,dateTime
492592,2019,9,29,0,EPA001,4.0,2019-09-29 00:00:00
492593,2019,9,29,0,EPA002,4.0,2019-09-29 00:00:00
492594,2019,9,29,0,EPA003,4.0,2019-09-29 00:00:00
492595,2019,9,29,0,EPA004,1.0,2019-09-29 00:00:00
492596,2019,9,29,0,EPA005,5.0,2019-09-29 00:00:00


In [9]:
station_num = len(np.unique(df_observation['stationID'])) ##測站總數

## 進行時間缺漏值填補

In [10]:
stations = list(pd.unique(df_observation['stationID']))

In [11]:
print('------observation 補缺失日期開始------')
df_observation['dateTime'] = pd.to_datetime(df_observation['dateTime']).apply(lambda x:int(time.mktime(x.timetuple())))
df_observation.index = np.arange(len(df_observation))
a = df_observation['dateTime'][0]
b = int(df_observation['dateTime'][-1:].values)
all_date = list(range(a,b,3600))+[b]
df_add = df_observation[: 1].copy()
for name in stations:
    #print('name', name)
    unnormal = list(df_observation[df_observation['stationID'] == name]['dateTime'])
    dates = list(set(all_date).difference(set(unnormal)))
    #print('date',len(dates))
    
    for date in dates:
        df_add['dateTime'] = date
        df_add['stationID'] = name
        df_add['PMf'] = np.nan
        df_observation = pd.concat([df_observation, df_add])
    df_observation.sort_values(by=['dateTime','stationID'], inplace = True)


------observation 補缺失日期開始------


In [12]:
 df_observation['dateTime'] = df_observation['dateTime'].apply(lambda x:time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(x)))
df_observation['month'] = pd.to_datetime(df_observation['dateTime']).dt.month
df_observation['year'] = pd.to_datetime(df_observation['dateTime']).dt.year
df_observation['day'] = pd.to_datetime(df_observation['dateTime']).dt.day
df_observation['hour'] = pd.to_datetime(df_observation['dateTime']).dt.hour

df_observation.tail()

,year,month,day,hour,stationID,PMf,dateTime
58603,2019,10,31,23,EPA071,37.0,2019-10-31 23:00:00
58604,2019,10,31,23,EPA072,43.0,2019-10-31 23:00:00
58605,2019,10,31,23,EPA080,4.0,2019-10-31 23:00:00
58606,2019,10,31,23,EPA102,19.0,2019-10-31 23:00:00
58607,2019,10,31,23,EPA110,27.0,2019-10-31 23:00:00


### 檢查補植後結果 1:補值成功 ; 不為1:補值失敗

In [13]:
if len(np.unique(pd.value_counts(df_observation['stationID'])))==1:
    print("observation補缺失日期成功")
else:
    print("observation補缺失日期失敗")

observation補缺失日期成功


In [14]:
df_observation.to_csv(file_name, sep='\t', index=False)  ##

In [15]:
df_observation['date'] = df_observation['dateTime'].apply(lambda x:time.strftime("%Y-%m-%d",time.strptime(x,'%Y-%m-%d %H:%M:%S')))

## 轉為一天為單位的矩陣

In [16]:
one_day_df = np.array(df_observation['PMf']).reshape(-1, station_num * 24)
one_day_df

array([[ 4.,  4.,  4., ...,  3., 30., 11.],
       [ 8.,  8.,  7., ...,  7.,  9., 13.],
       [ 6.,  5.,  8., ...,  3., 39., 23.],
       ...,
       [ 6.,  6.,  3., ...,  0., 61., 18.],
       [19., 19., 24., ...,  7., 31., 24.],
       [27., 26., 40., ...,  4., 19., 27.]])

In [17]:
index = pd.unique(df_observation['date'] ).astype(str)
observation_z = pd.DataFrame(columns=np.arange(station_num*24), data = one_day_df,index = index)
observation_y1 =  observation_z.copy()
observation_z.tail()

,0,1,2,3,4,5,6,7,8,9,...,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775
2019-10-27,15.0,20.0,14.0,15.0,15.0,19.0,15.0,20.0,24.0,20.0,...,4.0,26.0,9.0,23.0,8.0,27.0,33.0,13.0,10.0,13.0
2019-10-28,8.0,9.0,8.0,5.0,11.0,10.0,10.0,14.0,15.0,12.0,...,4.0,16.0,9.0,13.0,3.0,14.0,21.0,11.0,2.0,7.0
2019-10-29,6.0,6.0,3.0,3.0,4.0,2.0,5.0,8.0,5.0,6.0,...,17.0,25.0,17.0,21.0,19.0,25.0,25.0,0.0,61.0,18.0
2019-10-30,19.0,19.0,24.0,15.0,18.0,7.0,12.0,12.0,21.0,18.0,...,12.0,42.0,29.0,45.0,25.0,61.0,60.0,7.0,31.0,24.0
2019-10-31,27.0,26.0,40.0,24.0,29.0,32.0,29.0,32.0,34.0,27.0,...,25.0,29.0,21.0,32.0,20.0,37.0,43.0,4.0,19.0,27.0


## 將PMF=NAN進行補值

### 引入R的套件

In [18]:
#%load_ext rpy2.ipython

In [19]:
robjects.r("install.packages('devtools')")

robjects.r('source("https://bioconductor.org/biocLite.R")')
r('''devtools::install_github("marchtaylor/sinkr")''')

0


In [20]:
devtools = importr('devtools')
sinkr = importr('sinkr')

In [21]:
numpy2ri.activate()
pandas2ri.activate()
rpy2_observation_z = pandas2ri.py2ri(observation_z)
robjects.r.assign('rpy2_observation_z', rpy2_observation_z)

X0,X1,X2,...,X1773,X1774,X1775
4.000000,4.000000,4.000000,...,3.000000,30.000000,11.000000
8.000000,8.000000,7.000000,...,7.000000,9.000000,13.000000
6.000000,5.000000,8.000000,...,3.000000,39.000000,23.000000
10.000000,17.000000,9.000000,...,12.000000,62.000000,52.000000
...,...,...,...,...,...,...
8.000000,9.000000,8.000000,...,11.000000,2.000000,7.000000
6.000000,6.000000,3.000000,...,0.000000,61.000000,18.000000
19.000000,19.000000,24.000000,...,7.000000,31.000000,24.000000
27.000000,26.000000,40.000000,...,4.000000,19.000000,27.000000


In [22]:
r('''
rpy2_observation_z[is.na(rpy2_observation_z)]<- NaN
y <- data.matrix(rpy2_observation_z)
Ya <- dineof(y)$Xa
observation_z<-data.frame(Ya)
head(observation_z)
''')

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\rpy2-2.9.4-py3.6-win-amd64.egg\rpy2\robjects\pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X1766,X1767,X1768,X1769,X1770,X1771,X1772,X1773,X1774,X1775
0,4.0,4.0,4.0,1.0,5.0,1.0,6.0,6.0,8.0,5.0,...,8.000000,18.0,11.0,23.0,6.0,18.0,29.0,3.0,30.0,11.000000
1,8.0,8.0,7.0,5.0,10.0,4.0,11.0,9.0,10.0,7.0,...,1.000000,18.0,9.0,3.0,4.0,12.0,5.0,7.0,9.0,13.000000
2,6.0,5.0,8.0,1.0,5.0,10.0,7.0,4.0,6.0,7.0,...,3.000000,26.0,13.0,12.0,7.0,33.0,9.0,3.0,39.0,23.000000
3,10.0,17.0,9.0,5.0,13.0,16.0,16.0,8.0,9.0,15.0,...,8.000000,43.0,59.0,37.0,9.0,48.0,41.0,12.0,62.0,52.000000
4,25.0,23.0,18.0,8.0,36.0,30.0,26.0,16.0,43.0,43.0,...,23.000000,53.0,31.0,28.0,36.0,21.0,32.0,8.0,38.0,42.000000
5,27.0,41.0,23.0,21.0,31.0,34.0,37.0,34.0,45.0,38.0,...,18.072161,26.0,36.0,40.0,9.0,14.0,59.0,17.0,22.0,43.372483


## 補值過後的observation

#### R dataframe 轉換 pandas

In [23]:
r.data('observation_z')
df_nomissing = pd.DataFrame(r['observation_z'])
df_nomissing.tail()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X1766,X1767,X1768,X1769,X1770,X1771,X1772,X1773,X1774,X1775
28,15.0,20.0,14.0,15.0,15.0,19.0,15.0,20.0,24.0,20.0,...,4.0,26.0,9.0,23.0,8.0,27.0,33.0,13.0,10.0,13.0
29,8.0,9.0,8.0,5.0,11.0,10.0,10.0,14.0,15.0,12.0,...,4.0,16.0,9.0,13.0,3.0,14.0,21.0,11.0,2.0,7.0
30,6.0,6.0,3.0,3.0,4.0,2.0,5.0,8.0,5.0,6.0,...,17.0,25.0,17.0,21.0,19.0,25.0,25.0,0.0,61.0,18.0
31,19.0,19.0,24.0,15.0,18.0,7.0,12.0,12.0,21.0,18.0,...,12.0,42.0,29.0,45.0,25.0,61.0,60.0,7.0,31.0,24.0
32,27.0,26.0,40.0,24.0,29.0,32.0,29.0,32.0,34.0,27.0,...,25.0,29.0,21.0,32.0,20.0,37.0,43.0,4.0,19.0,27.0


In [24]:
index = pd.unique(df_observation['date'])
observation_z = pd.DataFrame(columns=np.arange(station_num*time_step), data = np.nan, index = index)
observation_z.head()

,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-09-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
j = 1
df_nomissing = np.array(df_nomissing)
for i in range(len(observation_z)):
    try:
        observation_z.iloc[i,:]  = np.vstack(df_nomissing[i + np.arange(time_interval)]).flatten()
    except:
        observation_z.iloc[i,:]  = np.vstack([df_nomissing[i + np.arange(time_interval-j)],np.zeros_like(df_nomissing[:j]) * np.nan]).flatten()
        j = j + 1
observation_z.tail()

,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-10-27,15.0,20.0,14.0,15.0,15.0,19.0,15.0,20.0,24.0,20.0,...,17.0,25.0,17.0,21.0,19.0,25.0,25.0,0.0,61.0,18.0
2019-10-28,8.0,9.0,8.0,5.0,11.0,10.0,10.0,14.0,15.0,12.0,...,12.0,42.0,29.0,45.0,25.0,61.0,60.0,7.0,31.0,24.0
2019-10-29,6.0,6.0,3.0,3.0,4.0,2.0,5.0,8.0,5.0,6.0,...,25.0,29.0,21.0,32.0,20.0,37.0,43.0,4.0,19.0,27.0
2019-10-30,19.0,19.0,24.0,15.0,18.0,7.0,12.0,12.0,21.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-31,27.0,26.0,40.0,24.0,29.0,32.0,29.0,32.0,34.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Observation_y

In [26]:
observation_y = pd.DataFrame(columns=np.arange(station_num * time_step), data = np.nan,index = index)
observation_y1 = np.array(observation_y1)
j = 1
df_nomissing = np.array(df_nomissing)
for i in range(len(observation_z)):
    try:
        observation_y.iloc[i,:]  = np.vstack(observation_y1[i + np.arange(time_interval)]).flatten()
    except:
        observation_y.iloc[i,:]  = np.vstack([observation_y1[i + np.arange(time_interval-j)],np.zeros_like(observation_y1[:j]) * np.nan]).flatten()
        j = j + 1
observation_y.tail()

,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-10-27,15.0,20.0,14.0,15.0,15.0,19.0,15.0,20.0,24.0,20.0,...,17.0,25.0,17.0,21.0,19.0,25.0,25.0,0.0,61.0,18.0
2019-10-28,8.0,9.0,8.0,5.0,11.0,10.0,10.0,14.0,15.0,12.0,...,12.0,42.0,29.0,45.0,25.0,61.0,60.0,7.0,31.0,24.0
2019-10-29,6.0,6.0,3.0,3.0,4.0,2.0,5.0,8.0,5.0,6.0,...,25.0,29.0,21.0,32.0,20.0,37.0,43.0,4.0,19.0,27.0
2019-10-30,19.0,19.0,24.0,15.0,18.0,7.0,12.0,12.0,21.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-31,27.0,26.0,40.0,24.0,29.0,32.0,29.0,32.0,34.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ----------------------------------------------------------------------------------------

# Prediction

In [27]:
def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date,"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += timedelta(days=1)
        
    return date_list

In [28]:
def data_cleaning_pred(df):
    df['expdate'] = pd.to_datetime(df['expdate']).apply(lambda x:int(time.mktime(x.timetuple())))
    return df

In [29]:
name = 'stat_v' 
for i in ver:
    new_name = name + str(i) + '.txt'
    prediction = pd.read_table(new_name)
    if i == ver[0]:
        new_prediction = prediction
    else:
        new_prediction = pd.concat([new_prediction, prediction], axis=0, sort = False)

In [30]:
new_prediction = data_cleaning_pred(new_prediction)
file_name = 'corr_old_preciction_' + str(time_step) + '.txt'

try:
    old_prediction = pd.read_table(file_name)
    old_prediction = data_cleaning_pred(old_prediction)
    old_end = old_prediction['expdate'].iloc[-1] + 3600
    new_prediction = new_prediction[new_prediction['expdate']>=old_end]
    df_prediction = pd.concat([old_prediction, new_prediction], axis = 0, ignore_index=True)
    
except IOError:
    df_prediction = new_prediction
    
df_prediction = df_prediction[df_prediction['expdate'].between(start - 3600*24, end_pre)] #end_pre - 3600 * 24
df_prediction['expdate'] = df_prediction['expdate'].apply(lambda x:time.strftime("%Y-%m-%d", time.localtime(x))) 
df_prediction.tail()

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,year,month,day,hour,expdate,tau,stationID,calPMf
595987,2019,11,4,23,2019-10-31,96,EPA084,0.159642
595988,2019,11,4,23,2019-10-31,96,EPA101,NaN
595989,2019,11,4,23,2019-10-31,96,EPA102,0.250123
595990,2019,11,4,23,2019-10-31,96,EPA104,0.521565
595991,2019,11,4,23,2019-10-31,96,EPA110,18.928040


## 拿取預測值、測站移除

In [31]:
df_prediction = df_prediction[df_prediction['tau']!=0]
df_prediction = df_prediction[df_prediction['tau'] <= time_step]

df_prediction = df_prediction[df_prediction['stationID']!='EPA101']
df_prediction = df_prediction[df_prediction['stationID']!='EPA084']
df_prediction = df_prediction[df_prediction['stationID']!='EPA104']
df_prediction.head()

,year,month,day,hour,expdate,tau,stationID,calPMf
297240,2019,9,30,0,2019-09-29,1,EPA001,10.384020
297241,2019,9,30,0,2019-09-29,1,EPA002,12.222376
297242,2019,9,30,0,2019-09-29,1,EPA003,5.971256
297243,2019,9,30,0,2019-09-29,1,EPA004,11.441598
297244,2019,9,30,0,2019-09-29,1,EPA005,10.245286


In [32]:
df_prediction.tail()

,year,month,day,hour,expdate,tau,stationID,calPMf
594136,2019,11,3,23,2019-10-31,72,EPA071,51.187390
594137,2019,11,3,23,2019-10-31,72,EPA072,14.749970
594138,2019,11,3,23,2019-10-31,72,EPA080,0.360062
594141,2019,11,3,23,2019-10-31,72,EPA102,0.444094
594143,2019,11,3,23,2019-10-31,72,EPA110,17.873880


## dateTime為預測的時間

In [33]:
df_prediction['expdate'] = pd.to_datetime(df_prediction['expdate']).apply(lambda x:int(time.mktime(x.timetuple())))
df_prediction['expdate'] = df_prediction['expdate'].apply(lambda x:time.strftime("%Y-%m-%d",time.localtime(x))) 
df_prediction.head()

,year,month,day,hour,expdate,tau,stationID,calPMf
297240,2019,9,30,0,2019-09-29,1,EPA001,10.384020
297241,2019,9,30,0,2019-09-29,1,EPA002,12.222376
297242,2019,9,30,0,2019-09-29,1,EPA003,5.971256
297243,2019,9,30,0,2019-09-29,1,EPA004,11.441598
297244,2019,9,30,0,2019-09-29,1,EPA005,10.245286


## 日期、時間缺漏值進行填補

In [34]:
def otherStyleTime(timestamp):
    timeArray = time.localtime(timestamp)
    date = time.strftime("%Y-%m-%d", timeArray)
    return(date)

In [35]:
start_pre = otherStyleTime(start)
end_pre = otherStyleTime(end_pre )

In [36]:
stations = df_prediction['stationID'].unique()
all_expdate = getEveryDay(start_pre, end_pre)
print('------prediction 補缺失日期開始------')
for name in stations:
    
    # 補一整天皆缺失的值
    unnormal = list(df_prediction[df_prediction['stationID'] == name]['expdate'])
    dates = list(set(all_expdate).difference(set(unnormal)))

    for date in dates:
        df_add = df_prediction[: time_step].copy()
        df_add['expdate'] = date
        df_add['hour'] = list(np.arange(24))*time_interval
        df_add['stationID'] = name
        df_add['tau'] = np.arange(time_step) + 1
        
        day = []
        month = []
        year = []
        for t in range(1, time_interval+1):
            day += [(pd.to_datetime(date) + timedelta(days=t)).day] * 24 
            month += [(pd.to_datetime(date) + timedelta(days=t)).month] * 24 
            year += [(pd.to_datetime(date) + timedelta(days=t)).year] * 24 
        df_add['day'] = day
        df_add['month'] = month
        df_add['year'] = year
        df_add['calPMf'] = np.nan
        df_prediction = pd.concat([df_prediction, df_add])

    df_prediction.sort_values(by=['expdate', 'tau', 'stationID'], inplace = True)

    # 補部分缺失的值
    if sum(pd.value_counts(df_prediction[df_prediction['stationID']==name]['expdate'])!=time_step)>0:
        #print(name, date)
        count = pd.value_counts(df_prediction[df_prediction['stationID']==name]['expdate']).values
        index = pd.value_counts(df_prediction[df_prediction['stationID']==name]['expdate']).index
        for expdate in index[count!=time_step]:
            all_hour = np.arange(time_step) + 1
            unnormal = df_prediction[df_prediction['stationID']==name][df_prediction['expdate']==expdate]['tau']
            taus = list(set(all_hour).difference(set(unnormal)))
            num = len(hours)

            df_add = df_prediction[: len(taus)].copy()
            df_add['expdate'] = expdate  
            df_add['hour'] = taus % 24 
            df_add['stationID'] = name
            df_add['tau'] = taus
            df_add['day'] = (pd.to_datetime(date) + timedelta(hours=23+taus)).day
            df_add['month'] = (pd.to_datetime(date) + timedelta(hours=23+taus)).month
            df_add['year'] = (pd.to_datetime(date) + timedelta(hours=23+taus)).year
            df_add['calPMf'] = np.nan
            df_prediction = pd.concat([df_prediction, df_add])

    df_prediction.sort_values(by=['expdate', 'tau', 'stationID'], inplace = True)
    print('stations:', name)

------prediction 補缺失日期開始------
stations: EPA001
stations: EPA002
stations: EPA003
stations: EPA004
stations: EPA005
stations: EPA006
stations: EPA007
stations: EPA008
stations: EPA009
stations: EPA010
stations: EPA011
stations: EPA012
stations: EPA013
stations: EPA014
stations: EPA015
stations: EPA016
stations: EPA017
stations: EPA018
stations: EPA019
stations: EPA020
stations: EPA021
stations: EPA022
stations: EPA023
stations: EPA024
stations: EPA025
stations: EPA026
stations: EPA027
stations: EPA028
stations: EPA029
stations: EPA030
stations: EPA031
stations: EPA032
stations: EPA033
stations: EPA034
stations: EPA035
stations: EPA036
stations: EPA037
stations: EPA038
stations: EPA039
stations: EPA040
stations: EPA041
stations: EPA042
stations: EPA043
stations: EPA044
stations: EPA045
stations: EPA046
stations: EPA047
stations: EPA048
stations: EPA049
stations: EPA050
stations: EPA051
stations: EPA052
stations: EPA053
stations: EPA054
stations: EPA056
stations: EPA057
stations: EPA058


In [37]:
if len(np.unique(pd.value_counts(df_prediction['stationID'])))==1:
    print("prediction補缺失日期成功")
else:
    print("prediction補缺失日期失敗")

prediction補缺失日期成功


In [38]:
df_prediction = df_prediction[['expdate','stationID','year','month','day','hour','tau','calPMf']]
df_prediction.head()

,expdate,stationID,year,month,day,hour,tau,calPMf
297240,2019-09-29,EPA001,2019,9,30,0,1,10.384020
297241,2019-09-29,EPA002,2019,9,30,0,1,12.222376
297242,2019-09-29,EPA003,2019,9,30,0,1,5.971256
297243,2019-09-29,EPA004,2019,9,30,0,1,11.441598
297244,2019-09-29,EPA005,2019,9,30,0,1,10.245286


In [39]:
df_prediction.to_csv(file_name, sep='\t', index=False)

## 將tau = 1的日期設為index

In [40]:
df_pred = pd.DataFrame(np.array(df_prediction['calPMf']).reshape([-1, time_step * station_num]))
df_pred.index = pd.to_datetime(df_prediction['expdate'].unique(), errors='coerce') + timedelta(days=1)
index = df_pred.index 
df_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-09-30,10.384020,12.222376,5.971256,11.441598,10.245286,14.770390,14.752220,16.288942,15.054422,9.530121,...,6.551325,6.648709,6.888054,15.446540,6.128881,71.70889,2.563082,0.654434,NaN,8.045380
2019-10-01,2.303247,1.167996,3.119260,1.364694,2.355424,3.042923,3.324206,9.170274,2.758917,1.507425,...,8.073354,9.741949,19.012670,36.024790,13.535630,46.60602,3.803456,0.691928,1.578971,1.534836
2019-10-02,18.529622,16.354433,18.980730,14.328082,18.858808,20.995500,22.920832,18.492727,28.264308,17.527233,...,8.417409,23.461110,33.428940,11.227320,32.616840,36.36791,3.497346,0.254489,34.839195,29.089000
2019-10-03,25.217936,27.691776,26.745528,24.618408,33.900899,35.589010,33.690254,38.341235,34.809658,29.657188,...,7.866346,47.601430,16.523700,1.502065,15.735460,147.13415,1.472838,0.353094,164.752345,13.292680
2019-10-04,38.854236,36.755066,35.572574,34.110252,48.683100,52.354600,49.098145,44.835977,43.202534,41.142932,...,2.602429,5.945044,12.129320,1.327380,7.761159,21.69153,2.312886,0.734246,54.006198,20.799630


In [41]:
df_pred.tail()

,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-10-28,13.427509,13.446619,12.398126,14.050098,16.355950,20.087620,16.750780,18.338449,19.555760,16.339273,...,8.265961,7.216942,7.790610,14.271270,4.566690,119.616840,4.381921,0.757728,11.389323,5.244024
2019-10-29,5.632875,4.230715,0.269621,2.059012,11.498530,11.264280,6.891853,10.196930,4.680984,2.894081,...,3.410336,7.111055,10.344857,65.649088,9.054913,26.173115,16.586438,1.491971,2.582207,9.006240
2019-10-30,1.289931,19.258952,0.432383,14.901676,14.629965,15.920009,17.553282,19.671755,11.698523,16.916361,...,2.414034,6.646315,4.300676,23.151850,3.250110,26.905080,18.146720,0.174710,0.259247,4.808150
2019-10-31,1.264438,29.788514,0.255462,22.720576,12.146773,22.353600,29.110149,34.981333,34.254231,25.641742,...,3.009589,3.709717,7.282560,23.191280,3.551979,36.680530,26.542503,0.535734,0.256727,5.043830
2019-11-01,2.259185,19.170722,0.316616,20.142869,9.882625,20.470040,20.447631,22.565321,22.329302,19.088418,...,3.513030,6.379925,11.479990,27.022449,3.002634,51.187390,14.749970,0.360062,0.444094,17.873880


## 將一整天為NAN的值刪掉

In [42]:
df_pred = df_pred.dropna(axis=0, how='all')
index = df_pred.index
df_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-09-30,10.384020,12.222376,5.971256,11.441598,10.245286,14.770390,14.752220,16.288942,15.054422,9.530121,...,6.551325,6.648709,6.888054,15.446540,6.128881,71.70889,2.563082,0.654434,NaN,8.045380
2019-10-01,2.303247,1.167996,3.119260,1.364694,2.355424,3.042923,3.324206,9.170274,2.758917,1.507425,...,8.073354,9.741949,19.012670,36.024790,13.535630,46.60602,3.803456,0.691928,1.578971,1.534836
2019-10-02,18.529622,16.354433,18.980730,14.328082,18.858808,20.995500,22.920832,18.492727,28.264308,17.527233,...,8.417409,23.461110,33.428940,11.227320,32.616840,36.36791,3.497346,0.254489,34.839195,29.089000
2019-10-03,25.217936,27.691776,26.745528,24.618408,33.900899,35.589010,33.690254,38.341235,34.809658,29.657188,...,7.866346,47.601430,16.523700,1.502065,15.735460,147.13415,1.472838,0.353094,164.752345,13.292680
2019-10-04,38.854236,36.755066,35.572574,34.110252,48.683100,52.354600,49.098145,44.835977,43.202534,41.142932,...,2.602429,5.945044,12.129320,1.327380,7.761159,21.69153,2.312886,0.734246,54.006198,20.799630


## 將PMF=NAN進行補值

In [43]:
rpy2_new_prediction = pandas2ri.py2ri(df_pred)
robjects.r.assign('rpy2_new_prediction', rpy2_new_prediction)

X0,X1,X2,...,X5325,X5326,X5327
10.384020,12.222376,5.971256,...,0.654434,nan,8.045380
2.303247,1.167996,3.119260,...,0.691928,1.578971,1.534836
18.529622,16.354433,18.980730,...,0.254489,34.839195,29.089000
25.217936,27.691776,26.745528,...,0.353094,164.752345,13.292680
...,...,...,...,...,...,...
5.632875,4.230715,0.269621,...,1.491971,2.582207,9.006240
1.289931,19.258952,0.432383,...,0.174710,0.259247,4.808150
1.264438,29.788514,0.255462,...,0.535734,0.256727,5.043830
2.259185,19.170722,0.316616,...,0.360062,0.444094,17.873880


In [44]:
r('''
rpy2_new_prediction[is.na(rpy2_new_prediction)]<- NaN
y <- data.matrix(rpy2_new_prediction)
Ya <- dineof(y)$Xa
new_prediction<-data.frame(Ya)
head(new_prediction)
''')

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X5318,X5319,X5320,X5321,X5322,X5323,X5324,X5325,X5326,X5327
0,10.384020,12.222376,5.971256,11.441598,10.245286,14.770390,14.752220,16.288942,15.054422,9.530121,...,6.551325,6.648709,6.888054,15.446540,6.128881,71.70889,2.563082,0.654434,16.030363,8.045380
1,2.303247,1.167996,3.119260,1.364694,2.355424,3.042923,3.324206,9.170274,2.758917,1.507425,...,8.073354,9.741949,19.012670,36.024790,13.535630,46.60602,3.803456,0.691928,1.578971,1.534836
2,18.529622,16.354433,18.980730,14.328082,18.858808,20.995500,22.920832,18.492727,28.264308,17.527233,...,8.417409,23.461110,33.428940,11.227320,32.616840,36.36791,3.497346,0.254489,34.839195,29.089000
3,25.217936,27.691776,26.745528,24.618408,33.900899,35.589010,33.690254,38.341235,34.809658,29.657188,...,7.866346,47.601430,16.523700,1.502065,15.735460,147.13415,1.472838,0.353094,164.752345,13.292680
4,38.854236,36.755066,35.572574,34.110252,48.683100,52.354600,49.098145,44.835977,43.202534,41.142932,...,2.602429,5.945044,12.129320,1.327380,7.761159,21.69153,2.312886,0.734246,54.006198,20.799630
5,17.153744,17.063243,13.142354,15.819230,24.312800,26.722910,25.678790,22.525031,21.538990,15.539159,...,17.842642,9.017360,6.484012,6.350275,6.738180,95.40394,1.535621,0.550834,32.301083,25.989810


In [45]:
r.data('new_prediction')
new_prediction = pd.DataFrame(r['new_prediction'])
new_prediction.index = index
new_prediction.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X5318,X5319,X5320,X5321,X5322,X5323,X5324,X5325,X5326,X5327
2019-09-30,10.384020,12.222376,5.971256,11.441598,10.245286,14.770390,14.752220,16.288942,15.054422,9.530121,...,6.551325,6.648709,6.888054,15.446540,6.128881,71.70889,2.563082,0.654434,16.030363,8.045380
2019-10-01,2.303247,1.167996,3.119260,1.364694,2.355424,3.042923,3.324206,9.170274,2.758917,1.507425,...,8.073354,9.741949,19.012670,36.024790,13.535630,46.60602,3.803456,0.691928,1.578971,1.534836
2019-10-02,18.529622,16.354433,18.980730,14.328082,18.858808,20.995500,22.920832,18.492727,28.264308,17.527233,...,8.417409,23.461110,33.428940,11.227320,32.616840,36.36791,3.497346,0.254489,34.839195,29.089000
2019-10-03,25.217936,27.691776,26.745528,24.618408,33.900899,35.589010,33.690254,38.341235,34.809658,29.657188,...,7.866346,47.601430,16.523700,1.502065,15.735460,147.13415,1.472838,0.353094,164.752345,13.292680
2019-10-04,38.854236,36.755066,35.572574,34.110252,48.683100,52.354600,49.098145,44.835977,43.202534,41.142932,...,2.602429,5.945044,12.129320,1.327380,7.761159,21.69153,2.312886,0.734246,54.006198,20.799630


In [47]:
new_prediction.tail()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X5318,X5319,X5320,X5321,X5322,X5323,X5324,X5325,X5326,X5327
2019-10-28,13.427509,13.446619,12.398126,14.050098,16.355950,20.087620,16.750780,18.338449,19.555760,16.339273,...,8.265961,7.216942,7.790610,14.271270,4.566690,119.616840,4.381921,0.757728,11.389323,5.244024
2019-10-29,5.632875,4.230715,0.269621,2.059012,11.498530,11.264280,6.891853,10.196930,4.680984,2.894081,...,3.410336,7.111055,10.344857,65.649088,9.054913,26.173115,16.586438,1.491971,2.582207,9.006240
2019-10-30,1.289931,19.258952,0.432383,14.901676,14.629965,15.920009,17.553282,19.671755,11.698523,16.916361,...,2.414034,6.646315,4.300676,23.151850,3.250110,26.905080,18.146720,0.174710,0.259247,4.808150
2019-10-31,1.264438,29.788514,0.255462,22.720576,12.146773,22.353600,29.110149,34.981333,34.254231,25.641742,...,3.009589,3.709717,7.282560,23.191280,3.551979,36.680530,26.542503,0.535734,0.256727,5.043830
2019-11-01,2.259185,19.170722,0.316616,20.142869,9.882625,20.470040,20.447631,22.565321,22.329302,19.088418,...,3.513030,6.379925,11.479990,27.022449,3.002634,51.187390,14.749970,0.360062,0.444094,17.873880


## Prediction 完成

## 將 Observation 對齊 Prediction 日期

In [48]:
new_prediction.index = (new_prediction.index).astype(str)
observation_z = pd.concat([observation_z, new_prediction], axis=1, join_axes=[new_prediction.index]).iloc[:, :time_step*station_num]
observation_z.tail()

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-10-28,8.0,9.0,8.0,5.0,11.0,10.0,10.0,14.0,15.0,12.0,...,12.0,42.0,29.0,45.0,25.0,61.0,60.0,7.0,31.0,24.0
2019-10-29,6.0,6.0,3.0,3.0,4.0,2.0,5.0,8.0,5.0,6.0,...,25.0,29.0,21.0,32.0,20.0,37.0,43.0,4.0,19.0,27.0
2019-10-30,19.0,19.0,24.0,15.0,18.0,7.0,12.0,12.0,21.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-31,27.0,26.0,40.0,24.0,29.0,32.0,29.0,32.0,34.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
new_prediction.index = (new_prediction.index).astype(str)
observation_y = pd.concat([observation_y, new_prediction], axis=1, join_axes=[new_prediction.index]).iloc[:, :time_step * station_num]
observation_y.tail()

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


,0,1,2,3,4,5,6,7,8,9,...,5318,5319,5320,5321,5322,5323,5324,5325,5326,5327
2019-10-28,8.0,9.0,8.0,5.0,11.0,10.0,10.0,14.0,15.0,12.0,...,12.0,42.0,29.0,45.0,25.0,61.0,60.0,7.0,31.0,24.0
2019-10-29,6.0,6.0,3.0,3.0,4.0,2.0,5.0,8.0,5.0,6.0,...,25.0,29.0,21.0,32.0,20.0,37.0,43.0,4.0,19.0,27.0
2019-10-30,19.0,19.0,24.0,15.0,18.0,7.0,12.0,12.0,21.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-31,27.0,26.0,40.0,24.0,29.0,32.0,29.0,32.0,34.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Observation 完成

# ----------------------------------------------------------------------------------------

## Model 4

In [52]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [53]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
tf.compat.v1.Session(config=config)

In [55]:
XData = np.array(new_prediction)[1:,:].astype('float32')
ZData = np.array(observation_z)[:-1,0:station_num*24].astype('float32')  #往前看24小時
YData = np.array(observation_y)[1:,:].astype('float32')

## 比較校正後的結果

In [59]:
new_model = tf.keras.models.load_model('model4_mne_softplus_1730_72.h5',compile=False)
y_hat = new_model([XData, ZData], training = False).numpy()

## 將資料整理回原格式

In [60]:
stations = np.unique(df_prediction['stationID'])
correction = list(y_hat.reshape(-1,1).flatten())
Pred = list(XData.flatten())
expdate = (pd.to_datetime(new_prediction.index)[1:] - timedelta(days=1)).astype(str).repeat(time_step*station_num)
stationID = list(stations) * time_step *len(XData)
tau = list((np.arange(time_step) + 1).repeat(len(stations))) * len(XData)
hour = np.array(tau) % 24
prediction = { 
             'expdate':expdate,
             'stationID':stationID,
             'tau':tau,
             'hour':hour,
             "PMf_ori": Pred,
             "PMf_corr": correction}
prediction_df = pd.DataFrame(prediction)
prediction_df['day'] = (pd.to_datetime(prediction_df['expdate']) + 
                        pd.TimedeltaIndex(23 + prediction_df['tau'], unit='h')).dt.day
prediction_df['month'] = (pd.to_datetime(prediction_df['expdate']) + 
                        pd.TimedeltaIndex(23 + prediction_df['tau'], unit='h')).dt.month
prediction_df['year'] = (pd.to_datetime(prediction_df['expdate']) + 
                        pd.TimedeltaIndex(23 + prediction_df['tau'], unit='h')).dt.year
prediction_df = prediction_df[['expdate','stationID','year','month','day','hour','tau','PMf_ori','PMf_corr']]
prediction_df['observation'] = np.array(observation_y[1:]).flatten(0)
prediction_df.head()

C:\Users\User\Anaconda3\envs\tf2_rpy2\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


,expdate,stationID,year,month,day,hour,tau,PMf_ori,PMf_corr,observation
0,2019-09-30,EPA001,2019,10,1,1,1,2.303247,4.804333,6.0
1,2019-09-30,EPA002,2019,10,1,1,1,1.167996,10.514338,5.0
2,2019-09-30,EPA003,2019,10,1,1,1,3.119260,4.602078,8.0
3,2019-09-30,EPA004,2019,10,1,1,1,1.364694,8.056275,1.0
4,2019-09-30,EPA005,2019,10,1,1,1,2.355424,7.557329,5.0


In [61]:
prediction_df.tail()

,expdate,stationID,year,month,day,hour,tau,PMf_ori,PMf_corr,observation
170491,2019-10-31,EPA071,2019,11,3,0,72,51.187389,23.884083,NaN
170492,2019-10-31,EPA072,2019,11,3,0,72,14.749970,20.816330,NaN
170493,2019-10-31,EPA080,2019,11,3,0,72,0.360062,3.954753,NaN
170494,2019-10-31,EPA102,2019,11,3,0,72,0.444094,11.671880,NaN
170495,2019-10-31,EPA110,2019,11,3,0,72,17.873880,20.255180,NaN


In [62]:
def error_print(df):
    def error(Y_hat, real):
        MNE = np.nanmean(abs((Y_hat[real!=0] - real[real!=0])/(real[real!=0]))) 
        MNB = np.nanmean((Y_hat[real!=0] - real[real!=0])/(real[real!=0]))
        return MNE, MNB
    df = df.dropna(subset=["observation"])  
                      
    MNE_corr, MNB_corr = error(df['PMf_corr'], df['observation'])  
    print('regression Mean Error = ', np.nanmean(abs(df['PMf_corr']-df['observation'])))
    print('regression Mean Normalized Error = ', MNE_corr)
    print('regression Mean Normalized Bias = ', MNB_corr)
    print('regression correlation coefficient = ', df.PMf_corr.corr(df.observation))
    print(' ')                  
    MNE_ori, MNB_ori = error(df['PMf_ori'], df['observation'])
    print('original Mean  Absolute Error = ', np.nanmean(abs(df['PMf_ori']-df['observation'])))
    print('original Mean Normalized Error = ', MNE_ori)
    print('original Mean Normalized Bias = ', MNB_ori)
    print('original correlation coefficient = ', df.PMf_ori.corr(df.observation))

In [63]:
error_print(prediction_df)

regression Mean Error =  11.744259283681062
regression Mean Normalized Error =  0.4705529310495956
regression Mean Normalized Bias =  -0.3349991388752119
regression correlation coefficient =  0.43129006924044305
 
original Mean  Absolute Error =  18.84130871119498
original Mean Normalized Error =  0.753791753658697
original Mean Normalized Bias =  -0.3755286386094416
original correlation coefficient =  0.15558535844284654


In [65]:
prediction_df.to_csv(sys.argv[5] + '/' + sys.argv[6])

IndexError: list index out of range